In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.SupportResistanceStrategy import SMAScanner
from ta.volatility import average_true_range

average_true_range(df_t['High'], df_t['Low'], df_t['Close'], window=14, fillna=False)/df_t['Close']

In [2]:
with open('files/watchlist.txt') as f:
    watchlist = f.read()

df_raw = yf.download(watchlist.split(' '), auto_adjust=False, interval='1d', period='6mo')

[*********************100%***********************]  80 of 80 completed


In [50]:
df = df_raw.stack([0,1], future_stack=True).reset_index().copy()
df.columns = ['Timestamp', 'Variable', 'Ticker', 'Value']
df = df.pivot(index=['Timestamp', 'Ticker'], columns='Variable', values='Value').reset_index().sort_values('Timestamp')
df.columns.name = None
# price = df.groupby('Ticker')['Close']
df_tick = df[df['Ticker'] == 'AAPL'].copy()

In [65]:
def identify_pivots(df:pd.DataFrame, threshold:float|str=0.05, use_high_low:bool=False) -> pd.DataFrame:
    if use_high_low:
        highs = df['High'].values
        lows = df['Low'].values
        candidate_price = highs[0]
    else:
        close = df['Close'].values
        candidate_price = close[0]

    if threshold == 'atr':
        atr = average_true_range(df['High'], df['Low'], df['Close'], window=14, fillna=False)
        atr_pct = atr / df['Close']
        atr_pct = atr_pct.replace(0, np.nan)
        threshold = atr_pct.fillna(value=atr_pct.mean()).values
    elif type(threshold) is float:
        threshold = [threshold] * len(df)

    zz_price = [np.nan] * len(df)
    zz_type = [np.nan] * len(df)
    last_pivot_idx = 0
    last_pivot_price = highs[0] if use_high_low else close[0]
    candidate_extreme_idx = 0
    candidate_extreme_price = highs[0] if use_high_low else close[0]
    direction = None # initialize

    for i in range(len(df)):
        thresh = threshold[i]
        if use_high_low:
            if direction == 'up':
                price = highs[i]
            elif direction == 'down':
                price = lows[i]
            else:
                price = highs[i]
        else:
            price = close[i]

        change_pct = (price - last_pivot_price) / last_pivot_price

        if direction is None:
            # No direction yet: look for first significant move
            if abs(change_pct) >= thresh:
                direction = 'up' if change_pct > 0 else 'down'
                candidate_extreme_idx = i
                candidate_extreme_price = price
                zz_price[last_pivot_idx] = last_pivot_price
                zz_type[last_pivot_idx] = 'high' if direction == 'up' else 'low'

        elif direction == 'up':
            if price > candidate_extreme_price:
                candidate_extreme_price = price
                candidate_extreme_idx = i
            elif price < candidate_extreme_price * (1 - thresh):
                # Commit the previous high as a pivot
                zz_price[candidate_extreme_idx] = candidate_extreme_price
                zz_type[candidate_extreme_idx] = 'high'
                last_pivot_price = price
                last_pivot_idx = i
                candidate_extreme_price = price
                candidate_extreme_idx = i
                direction = 'down'

        elif direction == 'down':
            if price < candidate_extreme_price:
                candidate_extreme_price = price
                candidate_extreme_idx = i
            elif price > candidate_extreme_price * (1 + thresh):
                # Commit the previous low as a pivot
                zz_price[candidate_extreme_idx] = candidate_extreme_price
                zz_type[candidate_extreme_idx] = 'low'
                last_pivot_price = price
                last_pivot_idx = i
                candidate_extreme_price = price
                candidate_extreme_idx = i
                direction = 'up'

    df['Pivot Level'] = zz_price
    df['Pivot Type'] = zz_type
    df['Threshold'] = threshold
    return df

In [66]:
df = identify_pivots(df_tick, threshold='atr',use_high_low=True)

In [67]:
df.dropna()

,Timestamp,Ticker,Adj Close,Close,High,Low,Open,Volume,Pivot Level,Pivot Type,Threshold
0,2025-01-13,AAPL,233.835922,234.399994,234.669998,229.720001,233.529999,49630700.0,234.669998,low,0.031543
480,2025-01-22,AAPL,223.291367,223.830002,224.119995,219.789993,219.789993,64126500.0,219.789993,low,0.031543
1040,2025-01-31,AAPL,235.432083,236.000000,247.190002,233.440002,247.190002,101075100.0,247.190002,high,0.028768
1200,2025-02-04,AAPL,232.239777,232.800003,233.130005,226.649994,227.250000,45067300.0,226.649994,low,0.030069
2320,2025-02-25,AAPL,246.716461,247.039993,250.000000,244.910004,248.000000,48013300.0,250.000000,high,0.021794
3280,2025-03-13,AAPL,209.405396,209.679993,216.839996,208.419998,215.949997,61368300.0,208.419998,low,0.033738
4240,2025-03-31,AAPL,221.839096,222.130005,225.619995,216.229996,217.009995,65299300.0,225.619995,high,0.027699
4720,2025-04-08,AAPL,172.194199,172.419998,190.339996,169.210007,186.699997,120859500.0,169.210007,low,0.054965
5040,2025-04-14,AAPL,202.254776,202.520004,212.940002,201.160004,211.440002,101352900.0,212.940002,high,0.057353
5360,2025-04-21,AAPL,192.907028,193.160004,193.800003,189.809998,193.270004,46742500.0,189.809998,low,0.053137


In [ ]:

threshold = 0.05
use_high_low: bool = True



if use_high_low:
    highs = df_t['High'].values
    lows = df_t['Low'].values
    candidate_price = highs[0]
else:
    close = df_t['Close'].values
    candidate_price = close[0]

zz_price = [np.nan] * len(df_t)
zz_type = [np.nan] * len(df_t)
last_pivot_idx = 0
last_pivot_price = close[0]
candidate_extreme_idx = 0
candidate_extreme_price = close[0]
direction = None # initialize

for i in range(len(close)):
    price = (
            highs[i] if direction == 'up' and use_high_low else
            lows[i] if direction == 'down' and use_high_low else
            close[i]
        )
    change_pct = (price - last_pivot_price) / last_pivot_price

    if direction is None:
        # No direction yet: look for first significant move
        if abs(change_pct) >= threshold:
            direction = 'up' if change_pct > 0 else 'down'
            candidate_extreme_idx = i
            candidate_extreme_price = price
            zz_price[last_pivot_idx] = last_pivot_price
            zz_type[last_pivot_idx] = 'high' if direction == 'up' else 'low'

    elif direction == 'up':
        if price > candidate_extreme_price:
            candidate_extreme_price = price
            candidate_extreme_idx = i
        elif price < candidate_extreme_price * (1 - threshold):
            # Commit the previous high as a pivot
            zz_price[candidate_extreme_idx] = candidate_extreme_price
            zz_type[candidate_extreme_idx] = 'high'
            last_pivot_price = price
            last_pivot_idx = i
            candidate_extreme_price = price
            candidate_extreme_idx = i
            direction = 'down'

    elif direction == 'down':
        if price < candidate_extreme_price:
            candidate_extreme_price = price
            candidate_extreme_idx = i
        elif price > candidate_extreme_price * (1 + threshold):
            # Commit the previous low as a pivot
            zz_price[candidate_extreme_idx] = candidate_extreme_price
            zz_type[candidate_extreme_idx] = 'low'
            last_pivot_price = price
            last_pivot_idx = i
            candidate_extreme_price = price
            candidate_extreme_idx = i
            direction = 'up'

# Later

In [ ]:
with open('cfg.txt') as f:

    token = f.read()

In [ ]:
import requests

url = "https://api.orats.io/datav2/hist/dailies"

querystring = {"token":token,"ticker":"AAPL","tradeDate":"2019-11-29"}

response = requests.request("GET", url, params=querystring)

print(response.text)


In [ ]:
import json
with open('backtest.json', 'r') as file:
    backtest = json.load(file)

headers = {
    "Authorization": token
}

response = requests.post(
    "https://api.orats.io/backtest/submit",
    headers=headers,
    data=backtest
)

In [ ]:
print("Status Code:", response.status_code)
print("Response Body:", response.text)

In [ ]:
with open('files/watchlist.txt') as f:
    watchlist = f.read()

df_raw = yf.download(watchlist.split(' '), auto_adjust=False, period='2y')

In [ ]:
scanner = SMAScanner(sma_periods=[50, 200], return_window=20)
output = scanner.run_scan(df_raw) # type: ignore
output

In [ ]:
with open('files/watchlist.txt') as f:
    watchlist = f.read()

df_raw = yf.download(watchlist.split(' '), auto_adjust=False, interval='60m', period='3mo')

In [ ]:
df = df_raw.stack(level=[0,1], future_stack=True).reset_index()
df.columns=['Timestamp', 'Variable', 'Ticker', 'Value']
df

In [ ]:
# assert set(df.columns) >= {'Timestamp', 'Variable', 'Ticker', 'Value'}
df_wide = df.pivot(index=['Timestamp', 'Ticker'], columns='Variable', values='Value').reset_index()
results = {}

window = 33
buffer = 0.01
for ticker, df_ticker in df_wide.groupby('Ticker'):
    df_ticker = df_ticker.sort_values('Timestamp').copy()
    df_ticker.set_index("Timestamp", inplace=True)
    if 'Open' not in df_ticker or 'Close' not in df_ticker:
        continue
    df_ticker = df_ticker.dropna(subset=['Open', 'Close'])
    body_top = df_ticker[['Open', 'Close']].max(axis=1)
    body_bottom = df_ticker[['Open', 'Close']].min(axis=1)
    pivot_high = (body_top == body_top.rolling(window, center=True).max())
    pivot_low = (body_bottom == body_bottom.rolling(window, center=True).min())
    df_ticker['pivot_high'] = pivot_high
    df_ticker['pivot_low'] = pivot_low
    resistance = df_ticker.loc[df_ticker['pivot_high'], ['Open', 'Close']].max(axis=1).tolist()
    support = df_ticker.loc[df_ticker['pivot_low'], ['Open', 'Close']].min(axis=1).tolist()

    # Group close levels
    def group_levels(levels):
        levels = sorted(levels)
        grouped = []
        for lvl in levels:
            if not grouped or abs(lvl - grouped[-1]) / grouped[-1] > buffer:
                grouped.append(lvl)
        return grouped

    results[ticker] = {
        "support": group_levels(support),
        "resistance": group_levels(resistance)
    }


In [ ]:
df_wide.groupby('Ticker').resample('1d', on='Timestamp')['Close'].last().dropna()